# Library

In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import base64
import email
from datetime import datetime, timezone
import pytz 
import os
import pytz
import time
import asyncio
import nest_asyncio
from bleak import BleakClient
from asyncio import Queue

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
DEVICE_ADDRESS = "DF:0A:3D:9C:02:2D"
UART_TX_CHAR_UUID = "6e400002-b5a3-f393-e0a9-e50e24dcca9e"  # TX

# Gmail Function

In [ ]:
def mylatestEmails():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    try:  
        service = build('gmail', 'v1', credentials=creds)
        result = service.users().messages().list(userId='me', q='is:unread').execute()
        messages = result.get('messages', [])
        if messages:
            msg = service.users().messages().get(userId='me', id=messages[0]['id']).execute()
            msg_payload = msg['payload']
            headers = msg_payload['headers']
            # Convert the received time to EST timezone
            received_time = int(msg.get('internalDate')) / 1000  # Convert from milliseconds to seconds
            utc_time = datetime.fromtimestamp(received_time, tz=timezone.utc)
            est = pytz.timezone('US/Eastern')
            received_time = utc_time.replace(tzinfo=pytz.utc).astimezone(est).strftime('%Y-%m-%d %H:%M:%S')
            for h in headers:
                if h['name'] == 'From':
                    sender = h['value']
            # print(f"Sender: {sender}, Received Time: {received_time}")
            return sender, received_time
    except HttpError as error:
        print(f'An error occurred: {error}')
        return None

import time

def listen_for_latest_emails(interval=60):
    """
    Continuously listens for the latest email using mylatestEmails().

    Args:
        interval (int): Time interval in seconds between each check. Default is 60 seconds.
    """
    last_sender = None
    last_received_time = None
    try:
        while True:
            # print("Checking for the latest email...")
            result = mylatestEmails()
            if result:
                sender, received_time = result
                if sender != last_sender or received_time != last_received_time:
                    print(f"New Email - Sender: {sender}, Received Time: {received_time}")
                    last_sender = sender
                    last_received_time = received_time
            time.sleep(interval)
    except KeyboardInterrupt:
        print("Stopped listening for emails.") 

# Sender

In [ ]:
import asyncio
from bleak import BleakClient
import nest_asyncio

# Apply nest_asyncio to allow nested event loops in Jupyter Notebook
nest_asyncio.apply()

# Constants
DEVICE_ADDRESS = "DF:0A:3D:9C:02:2D"  # Your VisRing MAC address
UART_TX_CHAR_UUID = "6e400002-b5a3-f393-e0a9-e50e24dcca9e"  # TX: write to device

async def interactive_ble_uart():
    print(f"Connecting to VisRing at {DEVICE_ADDRESS}...")
    async with BleakClient(DEVICE_ADDRESS, timeout=60.0) as client:
        if not client.is_connected:
            print("Failed to connect!")
            return

        print("Connected. Type your message below.")
        print("Type 'exit' to disconnect and quit.\n")

        while True:
            message = input("Send > ")
            if message.lower() == "exit":
                print("Disconnecting...")
                break
            await client.write_gatt_char(UART_TX_CHAR_UUID, message.encode("utf-8"))
            print(f"Message sent: '{message}'")

await interactive_ble_uart()

In [ ]:
import asyncio
from bleak import BleakClient
import nest_asyncio

# Apply nest_asyncio to allow nested event loops in Jupyter Notebook
nest_asyncio.apply()

# Constants
DEVICE_ADDRESS = "DF:0A:3D:9C:02:2D"  # Your VisRing MAC address
UART_TX_CHAR_UUID = "6e400002-b5a3-f393-e0a9-e50e24dcca9e"  # TX: write to device

async def gmail_ble_uart(interval=60):
    print(f"Connecting to VisRing at {DEVICE_ADDRESS}...")
    async with BleakClient(DEVICE_ADDRESS, timeout=60.0) as client:
        if not client.is_connected:
            print("Failed to connect!")
            return

        print("Connected. Type your message below.")
        print("Type 'exit' to disconnect and quit.\n")

        last_sender = None
        last_received_time = None
        while True:
            result = mylatestEmails()
            if result:
                sender, received_time = result
                if sender != last_sender or received_time != last_received_time:
                    print(f"New Email - Sender: {sender}, Received Time: {received_time}")
                    message = sender
                    await client.write_gatt_char(UART_TX_CHAR_UUID, message.encode("utf-8"))
                    print(f"Message sent: '{message}'")
                    last_sender = sender
                    last_received_time = received_time
            time.sleep(interval)

await gmail_ble_uart(interval=5)

# Scan BLE

In [ ]:
import asyncio
from bleak import BleakScanner

async def scan_ble_devices(scan_duration: int = 5):
    print(f"Scanning for BLE devices for {scan_duration} seconds...\n")
    devices = await BleakScanner.discover(timeout=scan_duration)

    if not devices:
        print("No BLE devices found.")
        return

    print("Devices found:")
    for i, device in enumerate(devices):
        if device.name and device.name != "Unknown":  # Filter out devices with name 'Unknown'
            print(f"{i+1}. Name: {device.name}, "
                  f"Address: {device.address}, "
                  f"RSSI: {device.rssi} dBm")

# Example use:
await scan_ble_devices(10)  # Scans for 5 seconds
